# Create CIGALE data file for (VIKING+DES+CATWISE) sources

Select a subsample of VIKING+DES+CATWISE sources that have the same distribution as EMU sources in a given magnitude.

# Import Libraries

In [2]:
import numpy as np
from astropy.table import Table
import pandas as pd
import matplotlib.pyplot as plt
import missingno

# Define the directory

In [3]:
observatory = '/home/mfonseca/fastdisk'
my_computer = '/home/polaris/Lab_Astro/data/'

directory = my_computer

# Import data

## Import EMU+VIKING+DES+CATWISE crossmatch data

In [ ]:
emu_matched_no_mags_path = directory + 'cross_match/EMU_0102-32_DESY6GOLD_VIKING_CATWISE_noMags/EMU_0102-32_DESY6GOLD_VIKINGDR5_CATWISE_noMag_petro_2.fits'
emu_matched_no_mags = Table.read(emu_matched_no_mags_path).filled()
emu_matched_no_mags = emu_matched_no_mags.to_pandas()

match_mask = (emu_matched_no_mags['match_flag'] == 1) & (emu_matched_no_mags['p_i'] >= 0.1) & (emu_matched_no_mags['p_any'] >=0.83)
catalog_matches_noMags = emu_matched_no_mags[match_mask]
print(f'Number of objects in the catalog {len(catalog_matches_noMags)}')

# Here we select objects that have matches in all three surveys
full_match_mask = (catalog_matches_noMags['DESY6_coadd_object_id'] != -99) & (catalog_matches_noMags['VKG_sourceID'] != -99) & (catalog_matches_noMags['CAT_source_id'] != b'-99')
emu_catalog_matches_noMags_full = catalog_matches_noMags[full_match_mask]

# Ignore the warning
emu_catalog_matches_noMags_full['EMU_island_name'] = emu_catalog_matches_noMags_full['EMU_island_name'].str.decode('utf-8').str.strip(' ')

print(f'Number of objects in the catalog that have matches in all three surveys {len(emu_catalog_matches_noMags_full)}')

emu_catalog_matches_noMags_full.drop(columns=['DESY6_mag_auto_y_extcorr', 'DESY6_z_y_extcorr','VKG_magerr_petro_z', 'VKG_mag_petro_z_ab_extcorr'], inplace=True)
emu_catalog_matches_noMags_full.dropna(inplace=True)

print(f'Number of objects in the catalog that have matches in all three surveys and have full photometry {len(emu_catalog_matches_noMags_full)}')

## Import VIKING+DES+CATWISE crossmatch data

In [ ]:
other_matched_no_mags_path = directory + 'cross_match/without_EMU/DESY6_VKG_CAT_noMags/DESY6_VKG_CAT_noMags.fits'
other_matched_no_mags = Table.read(other_matched_no_mags_path).filled()
other_matched_no_mags = other_matched_no_mags.to_pandas()

# Match criteria from NWAY
match_mask = (other_matched_no_mags['match_flag'] == 1) & (other_matched_no_mags['p_i'] >= 0.1) & (other_matched_no_mags['p_any'] >=0.00)
other_catalog_matches_noMags = other_matched_no_mags[match_mask]
print(f'Number of objects in the catalog {len(other_catalog_matches_noMags)}')

# Here we select objects that have matches in all three surveys
full_match_mask = (catalog_matches_noMags['DESY6_coadd_object_id'] != -99) & (catalog_matches_noMags['VKG_sourceID'] != -99) & (catalog_matches_noMags['CAT_source_id'] != b'-99')
other_catalog_matches_noMags_full = other_catalog_matches_noMags[full_match_mask]

print(f'Number of objects in the catalog that have matches in all three surveys {len(other_catalog_matches_noMags_full)}')

# Remove objects with incomplete photometry
other_catalog_matches_noMags_full.dropna(inplace=True)

print(f'Number of objects in the catalog that have matches in all three surveys and have full photometry {len(other_catalog_matches_noMags_full)}')

## Remove EMU+VIKING+DES+CATWISE crossmatch data from VIKING+DES 

# Select a subsample of VIKING+DES+CATWISE with similar distribution to EMU+VIKING+DES+CATWISE

# Save the subsample to a CIGALE file